In [1]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import plotly.express as px
import plotly.io as pio
from scipy import stats
pio.renderers.default = "vscode"
pio.renderers
from plotly.graph_objs import *
import plotly.graph_objects as go

In [2]:
streaming_one = json.load(open("/Volumes/HD/GitHub/spotify_app/spotify_data/StreamingHistory0.json"))
streaming_two = json.load(open("/Volumes/HD/GitHub/spotify_app/spotify_data/StreamingHistory1.json"))

In [4]:
history_df = pd.DataFrame(columns=["endTime", "artistName", "trackName", "msPlayed"])

In [223]:
history_play_count = history_df.groupby("trackName")["trackName"].size().reset_index(name="count")

In [224]:
history_play_count = history_play_count.groupby('count').size().reset_index(name='number_of_songs')

In [225]:
plot_df = history_play_count.copy()

In [226]:
fig = px.scatter(x=plot_df["number_of_songs"], y=plot_df["count"], labels={"x":"Number of songs", "y":"Number of plays"}, color_discrete_sequence=['#1DB954'])
fig.update_layout(
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    title_font_color="white",
    title = {
        "text":"Do you attach to songs?",
        "x": 0.5,
        "xanchor":"center",
        "yanchor": "top"
    }
)
fig.update_xaxes(color="white")
fig.update_yaxes(color="white")
fig.show()

ValueError: Cannot accept list of column references or list of columns for both `x` and `y`.

In [5]:
dataframe = history_df.append(streaming_one, ignore_index=True)
dataframe = history_df.append(streaming_two, ignore_index=True)
dataframe["sec_played"] = dataframe["msPlayed"]/1000
dataframe = dataframe.drop(["msPlayed"], axis=1)

In [8]:
dataframe.head(10)

,endTime,artistName,trackName,sec_played
0,2022-06-14 15:29,Lorde,Solar Power,192.621
1,2022-06-14 15:31,Lorde,California,82.636
2,2022-06-14 22:23,Lorde,California,162.074
3,2022-06-14 22:28,Lorde,Stoned at the Nail Salon,266.432
4,2022-06-14 22:32,Lorde,Fallen Fruit,238.033
5,2022-06-14 22:35,Lorde,Secrets from a Girl (Who's Seen it All),218.659
6,2022-06-14 22:40,Lorde,The Man with the Axe,255.89
7,2022-06-14 22:42,Lorde,Dominoes,123.147
8,2022-06-14 22:44,Lorde,Big Star,167.044
9,2022-06-14 22:46,Lorde,Leader of a New Regime,93.414


In [372]:
dataframe.head()

,endTime,artistName,trackName,sec_played,week
0,2022-06-14 15:29,Lorde,Solar Power,192.621,2022-24
1,2022-06-14 15:31,Lorde,California,82.636,2022-24
2,2022-06-14 22:23,Lorde,California,162.074,2022-24
3,2022-06-14 22:28,Lorde,Stoned at the Nail Salon,266.432,2022-24
4,2022-06-14 22:32,Lorde,Fallen Fruit,238.033,2022-24


In [396]:
dataframe = dataframe[dataframe["endTime"].str.startswith("2022")]

In [ ]:
dataframe

In [397]:
# Convert date
dataframe['date'] = pd.to_datetime(dataframe['endTime'])
dataframe['date'] = pd.to_datetime(dataframe['date'].dt.date)
# Extract the hour and week from the datetime
dataframe['hour'] = pd.to_datetime(dataframe['endTime']).dt.hour
dataframe["week"] = pd.to_datetime(dataframe["endTime"]).dt.week
# Group the data by week and hour
df_grouped = dataframe.groupby(['week', 'hour']).size().reset_index(name='count')

<ipython-input-397-e2d06a62bce7>:6: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



In [398]:
weeks = range(1, 53)
hours = range(0, 24)
new_index = pd.MultiIndex.from_product([weeks, hours], names=['week', 'hour'])
df_plot = df_grouped.set_index(['week','hour']).reindex(new_index, fill_value=0).reset_index()

In [399]:
y_min = df_plot[df_plot["count"] != 0]["week"].min()
y_max = df_plot[df_plot["count"] != 0]["week"].max()

fig = go.Figure(data=go.Heatmap(
                   x=df_plot['hour'],
                   y=df_plot['week'],
                   z=df_plot['count'],
                   type='heatmap',
                   colorscale="viridis",
                   zmin=-1,
                   zmax=df_plot["count"].max(),
                   colorbar=dict(tickcolor="white", tickfont=dict(color="white")),
                   hovertemplate="Hour: %{x} <br>Week: %{y}<br>Number of plays: %{z}<extra></extra>"
          
))
fig.update_layout(xaxis_title="Hour of the day", 
                  yaxis_title="Week of the year", 
                  yaxis_range=(y_min + 1, y_max),
                  xaxis_tick0=0,
                  xaxis_dtick=3,
                  yaxis_dtick=3,
                  paper_bgcolor="rgba(0,0,0,0)",
                  plot_bgcolor="rgba(0,0,0,0)",
                  title_font_color="white",
                  title = {
                            "text":"When have you listened to music?",
                            "x": 0.5,
                            "xanchor":"center",
                            "yanchor": "top"}, 
                  showlegend=False)
fig.update_xaxes(color="white")
fig.update_yaxes(color="white")
fig.show()


In [200]:
fig = go.Figure(data=go.Histogram2d(
    x=df_grouped['hour'],
    y=df_grouped['week'],
    colorscale='Viridis',
    z=df_grouped['count'],
    colorbar=dict(tickcolor="white", tickfont=dict(color="white")),
    hovertemplate="Hour: %{x} <br>Week: %{y}<br>Number of plays: %{z}<extra></extra>",
    autobinx=False,
    xbins=dict(start=0, end=24, size=1),
    autobiny=False,
    ybins=dict(start=0,
               end=48,
               size=1)
))
# Set the labels for the x and y axis
fig.update_layout(xaxis_title="Hour", 
                  yaxis_title="Week of Year", 
                  paper_bgcolor="rgba(0,0,0,0)",
                  plot_bgcolor="rgba(0,0,0,0)",
                  title_font_color="white",
                  title = {
                            "text":"When have you listened to music?",
                            "x": 0.5,
                            "xanchor":"center",
                            "yanchor": "top"}, 
                  showlegend=False)
fig.update_xaxes(color="white")
fig.update_yaxes(color="white")

fig.show()